In [1]:
import os
from argparse import Namespace

import argparse
import os
import time
from pathlib import Path

import cv2
import torch
import torch.backends.cudnn as cudnn
from numpy import random

from models.experimental import attempt_load
from utils.datasets import LoadStreams, LoadImages
from utils.general import check_img_size, non_max_suppression, apply_classifier, scale_coords, xyxy2xywh, \
    plot_one_box, strip_optimizer, set_logging, increment_dir
from utils.torch_utils import select_device, load_classifier, time_synchronized

In [4]:
print(os.listdir("."))
print(os.getcwd())

['.amlignore', '.amlignore.amltmp', '.dockerignore', '.git', '.gitattributes', '.github', '.gitignore', '.ipynb_aml_checkpoints', '.ipynb_checkpoints', 'data', 'detect.py', 'detect.sh', 'detect_and_cut.ipynb', 'detect_and_cut.ipynb.amltmp', 'Dockerfile', 'evolve.txt', 'hubconf.py', 'LICENSE', 'model.ipynb', 'model.py', 'models', 'other_commands.txt', 'prepare_data.sh', 'README-ORIG.md', 'README.md', 'requirements.txt', 'runs', 'test.py', 'train.py', 'train_general.sh', 'train_price.sh', 'tutorial.ipynb', 'utils', 'via_to_coco.py', 'weights', 'yolov5s.pt', '__pycache__']
/mnt/batch/tasks/shared/LS_root/mounts/clusters/nc6-pm/code/Users/petr.michal/yolov5


In [3]:
from detect import detect

In [1]:
# save_dir, source, weights, view_img, save_txt, imgsz = \
#         Path(opt.save_dir), opt.source, opt.weights, opt.view_img, opt.save_txt, opt.img_size

In [7]:
opt = Namespace()
opt.weights = "../../../pricetag_reader/yolov5/weights/pricetag_small.pt"
opt.source = "../../../pricetag_reader/kaufland_photos"
# opt.weights = "pricetag_reader/yolov5/weights/pricetag_small.pt"  # product name and price model #"runs/train/exp8/weights/best.pt"
# opt.source = "../kaufland_photos"  #"data/images"
opt.conf_thres = 0.25
opt.iou_thres = 0.45
opt.save_txt = False
opt.view_img = False
opt.augment = False
opt.update = False
opt.device = "cpu"
opt.img_size = 640
opt.classes = None
opt.agnostic_nms = False
opt.save_img = False
opt.name = ""
# save only cut detections
opt.save_cut = True

webcam = False
save_dir = Path("../../../pricetag_reader/kaufland_photos_p") #Path('runs/detect')
# parser.add_argument('--source', type=str, default='data/images', help='source')  # file/folder, 0 for webcam
# parser.add_argument('--img-size', type=int, default=640, help='inference size (pixels)')
# parser.add_argument('--conf-thres', type=float, default=0.25, help='object confidence threshold')
# parser.add_argument('--iou-thres', type=float, default=0.45, help='IOU threshold for NMS')
# parser.add_argument('--device', default='', help='cuda device, i.e. 0 or 0,1,2,3 or cpu')
# parser.add_argument('--view-img', action='store_true', help='display results')
# parser.add_argument('--save-txt', action='store_true', help='save results to *.txt')
# parser.add_argument('--save-conf', action='store_true', help='save confidences in --save-txt labels')
# parser.add_argument('--save-dir', type=str, default='runs/detect', help='directory to save results')
# parser.add_argument('--name', default='', help='name to append to --save-dir: i.e. runs/{N} -> runs/{N}_{name}')
# parser.add_argument('--classes', nargs='+', type=int, help='filter by class: --class 0, or --class 0 2 3')
# parser.add_argument('--agnostic-nms', action='store_true', help='class-agnostic NMS')
# parser.add_argument('--augment', action='store_true', help='augmented inference')
# parser.add_argument('--update', action='store_true', help='update all models')

In [8]:
# Initialize
device = select_device(opt.device)
half = device.type != 'cpu'  # half precision only supported on CUDA

# Load model
model = attempt_load(opt.weights, map_location=opt.device)  # load FP32 model
imgsz = check_img_size(opt.img_size, s=model.stride.max())  # check img_size
if half:
    model.half()  # to FP16

# Second-stage classifier
classify = False
if classify:
    modelc = load_classifier(name='resnet101', n=2)  # initialize
    modelc.load_state_dict(torch.load('weights/resnet101.pt', map_location=device)['model'])  # load weights
    modelc.to(device).eval()

# Set Dataloader
#save_img = False # True
dataset = LoadImages(opt.source, img_size=opt.img_size)
print("{} files found".format(len(dataset.files)))

# Get names and colors
names = model.module.names if hasattr(model, 'module') else model.names
colors = [[random.randint(0, 255) for _ in range(3)] for _ in range(len(names))]
print("Done")

Fusing layers... 
90 files found
Done


In [10]:
# Directories
if save_dir == Path('runs/detect'):  # if default
    os.makedirs('runs/detect', exist_ok=True)  # make base
    save_dir = Path(increment_dir(save_dir / 'exp', opt.name))  # increment run
os.makedirs(save_dir / 'labels' if opt.save_txt else save_dir, exist_ok=True)  # make new dir


# Run inference
t0 = time.time()
img = torch.zeros((1, 3, opt.img_size, opt.img_size), device=device)  # init img
_ = model(img.half() if half else img) if device.type != 'cpu' else None  # run once

n_photos = 0
for path, img, im0s, vid_cap in dataset:
    print(path)
# if path.split("/")[-1] != "20200827_152613.jpg":
#     continue
#    print(path,"------")
    print(img.shape)
#     print(img[0,300:320,300:320])
    img = torch.from_numpy(img).to(device)
    img = img.half() if half else img.float()  # uint8 to fp16/32
    img /= 255.0  # 0 - 255 to 0.0 - 1.0
    if img.ndimension() == 3:
        img = img.unsqueeze(0)

    # uncomment this to process only given amount of photos  
#     if n_photos > 40:
#          break
    n_photos += 1
    
    # Inference
    t1 = time_synchronized()
    pred = model(img, augment=opt.augment)[0]

    # Apply NMS
    pred = non_max_suppression(pred, opt.conf_thres, opt.iou_thres, classes=opt.classes, agnostic=opt.agnostic_nms)
    t2 = time_synchronized()
    
    # Apply Classifier
    if classify:
        pred = apply_classifier(pred, modelc, img, im0s)


    # Process detections
    for i, det in enumerate(pred):  # detections per image
        cnt = 0
        if webcam:  # batch_size >= 1
            p, s, im0 = Path(path[i]), '%g: ' % i, im0s[i].copy()
        else:
            p, s, im0 = Path(path), '', im0s

        save_path = str(save_dir / p.name)
        txt_path = str(save_dir / 'labels' / p.stem) + ('_%g' % dataset.frame if dataset.mode == 'video' else '')
        s += '%gx%g ' % img.shape[2:]  # print string
        gn = torch.tensor(im0.shape)[[1, 0, 1, 0]]  # normalization gain whwh
        if det is not None and len(det):
            # Rescale boxes from img_size to im0 size
            det[:, :4] = scale_coords(img.shape[2:], det[:, :4], im0.shape).round()

            # Print results
            for c in det[:, -1].unique():
                n = (det[:, -1] == c).sum()  # detections per class
                s += '%g %ss, ' % (n, names[int(c)])  # add to string

            # Write results
            for *xyxy, conf, cls in reversed(det):
                if opt.save_txt:  # Write to file
                    xywh = (xyxy2xywh(torch.tensor(xyxy).view(1, 4)) / gn).view(-1).tolist()  # normalized xywh
                    line = (cls, *xywh, conf) if opt.save_conf else (cls, *xywh)  # label format
                    with open(txt_path + '.txt', 'a') as f:
                        f.write(('%g ' * len(line) + '\n') % line)

                if opt.save_img or opt.view_img:  # Add bbox to image
                    label = '%s %.2f' % (names[int(cls)], conf)
                    plot_one_box(xyxy, im0, label=label, color=colors[int(cls)], line_thickness=3)
                
                if opt.save_cut: # Cut and save detections
                    cut = im0[int(xyxy[1]):int(xyxy[3]),int(xyxy[0]):int(xyxy[2]),:]
                    cv2.imwrite("{}_{}.jpg".format(save_path.split(".jpg")[0], cnt), cut)
                cnt += 1
                    
                    
        # Print time (inference + NMS)
        print('%sDone. (%.3fs)' % (s, t2 - t1))
        break
        
        # Stream results
        if opt.view_img:
            cv2.imshow(p, im0)
            if cv2.waitKey(1) == ord('q'):  # q to quit
                raise StopIteration

        # Save results (image with detections)
        if opt.save_img:
            if dataset.mode == 'images':
                cv2.imwrite(save_path, im0)
            else:
                if vid_path != save_path:  # new video
                    vid_path = save_path
                    if isinstance(vid_writer, cv2.VideoWriter):
                        vid_writer.release()  # release previous video writer

                    fourcc = 'mp4v'  # output video codec
                    fps = vid_cap.get(cv2.CAP_PROP_FPS)
                    w = int(vid_cap.get(cv2.CAP_PROP_FRAME_WIDTH))
                    h = int(vid_cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
                    vid_writer = cv2.VideoWriter(save_path, cv2.VideoWriter_fourcc(*fourcc), fps, (w, h))
                vid_writer.write(im0)

image 1/90 /mnt/batch/tasks/shared/LS_root/mounts/clusters/nc6-pm/code/pricetag_reader/kaufland_photos/20200827_152613.jpg: /mnt/batch/tasks/shared/LS_root/mounts/clusters/nc6-pm/code/pricetag_reader/kaufland_photos/20200827_152613.jpg
(3, 640, 384)
640x384 19 prices, Done. (0.075s)
image 2/90 /mnt/batch/tasks/shared/LS_root/mounts/clusters/nc6-pm/code/pricetag_reader/kaufland_photos/IMG_20181212_171636.jpg: /mnt/batch/tasks/shared/LS_root/mounts/clusters/nc6-pm/code/pricetag_reader/kaufland_photos/IMG_20181212_171636.jpg
(3, 480, 640)
480x640 16 prices, Done. (0.091s)
image 3/90 /mnt/batch/tasks/shared/LS_root/mounts/clusters/nc6-pm/code/pricetag_reader/kaufland_photos/IMG_20181212_171641.jpg: /mnt/batch/tasks/shared/LS_root/mounts/clusters/nc6-pm/code/pricetag_reader/kaufland_photos/IMG_20181212_171641.jpg
(3, 480, 640)
480x640 7 prices, Done. (0.092s)
image 4/90 /mnt/batch/tasks/shared/LS_root/mounts/clusters/nc6-pm/code/pricetag_reader/kaufland_photos/IMG_20181212_171647.jpg: /mnt/

KeyboardInterrupt: 

In [19]:
det.shape[0]

1

In [52]:
from PIL import Image
#int(xyxy[0])
cut = im0[int(xyxy[1]):int(xyxy[3]),int(xyxy[0]):int(xyxy[2]),:]
Image.fromarray(cut)

'runs/detect/exp11/IMG_20200901_173032.jpg'

In [71]:
# "{}_{}.jpg".format(save_path.split(".jpg")[0], cnt)
#cv2.imwrite(cut, "{}_{}.jpg".format(save_path.split(".jpg")[0], cnt))
cut

array([[[232, 232, 232],
        [233, 233, 233],
        [233, 233, 233],
        ...,
        [231, 229, 229],
        [232, 230, 230],
        [234, 232, 232]],

       [[232, 232, 232],
        [232, 232, 232],
        [234, 234, 234],
        ...,
        [232, 230, 230],
        [233, 231, 231],
        [234, 232, 232]],

       [[231, 231, 231],
        [230, 230, 230],
        [233, 233, 233],
        ...,
        [234, 232, 232],
        [235, 233, 233],
        [235, 233, 233]],

       ...,

       [[195, 193, 192],
        [195, 193, 192],
        [197, 195, 194],
        ...,
        [238, 236, 236],
        [237, 235, 235],
        [236, 234, 234]],

       [[200, 198, 197],
        [200, 198, 197],
        [204, 202, 201],
        ...,
        [232, 230, 230],
        [232, 230, 230],
        [233, 231, 231]],

       [[219, 217, 216],
        [219, 217, 216],
        [220, 218, 217],
        ...,
        [217, 215, 214],
        [221, 219, 218],
        [223, 221, 220]]

In [21]:


# # Initialize
# set_logging()
# device = select_device(opt.device)
# half = device.type != 'cpu'  # half precision only supported on CUDA

# # Load model
# model = attempt_load(weights, map_location=device)  # load FP32 model
# imgsz = check_img_size(imgsz, s=model.stride.max())  # check img_size
# if half:
#     model.half()  # to FP16

# # Second-stage classifier
# classify = False
# if classify:
#     modelc = load_classifier(name='resnet101', n=2)  # initialize
#     modelc.load_state_dict(torch.load('weights/resnet101.pt', map_location=device)['model'])  # load weights
#     modelc.to(device).eval()

# # Set Dataloader
# vid_path, vid_writer = None, None
# if webcam:
#     view_img = True
#     cudnn.benchmark = True  # set True to speed up constant image size inference
#     dataset = LoadStreams(source, img_size=imgsz)
# else:
#     save_img = True
#     dataset = LoadImages(source, img_size=imgsz)

# # Get names and colors
# names = model.module.names if hasattr(model, 'module') else model.names
# colors = [[random.randint(0, 255) for _ in range(3)] for _ in range(len(names))]

# Run inference
t0 = time.time()
img = torch.zeros((1, 3, imgsz, imgsz), device=device)  # init img
_ = model(img.half() if half else img) if device.type != 'cpu' else None  # run once
for path, img, im0s, vid_cap in dataset:
    img = torch.from_numpy(img).to(device)
    img = img.half() if half else img.float()  # uint8 to fp16/32
    img /= 255.0  # 0 - 255 to 0.0 - 1.0
    if img.ndimension() == 3:
        img = img.unsqueeze(0)

    # Inference
    t1 = time_synchronized()
    pred = model(img, augment=opt.augment)[0]

    # Apply NMS
    pred = non_max_suppression(pred, opt.conf_thres, opt.iou_thres, classes=opt.classes, agnostic=opt.agnostic_nms)
    t2 = time_synchronized()

    # Apply Classifier
    if classify:
        pred = apply_classifier(pred, modelc, img, im0s)

    # Process detections
    for i, det in enumerate(pred):  # detections per image
        if webcam:  # batch_size >= 1
            p, s, im0 = Path(path[i]), '%g: ' % i, im0s[i].copy()
        else:
            p, s, im0 = Path(path), '', im0s

        save_path = str(save_dir / p.name)
        txt_path = str(save_dir / 'labels' / p.stem) + ('_%g' % dataset.frame if dataset.mode == 'video' else '')
        s += '%gx%g ' % img.shape[2:]  # print string
        gn = torch.tensor(im0.shape)[[1, 0, 1, 0]]  # normalization gain whwh
        if det is not None and len(det):
            # Rescale boxes from img_size to im0 size
            det[:, :4] = scale_coords(img.shape[2:], det[:, :4], im0.shape).round()

            # Print results
            for c in det[:, -1].unique():
                n = (det[:, -1] == c).sum()  # detections per class
                s += '%g %ss, ' % (n, names[int(c)])  # add to string

            # Write results
            for *xyxy, conf, cls in reversed(det):
                if save_txt:  # Write to file
                    xywh = (xyxy2xywh(torch.tensor(xyxy).view(1, 4)) / gn).view(-1).tolist()  # normalized xywh
                    line = (cls, *xywh, conf) if opt.save_conf else (cls, *xywh)  # label format
                    with open(txt_path + '.txt', 'a') as f:
                        f.write(('%g ' * len(line) + '\n') % line)

                if save_img or view_img:  # Add bbox to image
                    label = '%s %.2f' % (names[int(cls)], conf)
                    plot_one_box(xyxy, im0, label=label, color=colors[int(cls)], line_thickness=3)

        # Print time (inference + NMS)
        print('%sDone. (%.3fs)' % (s, t2 - t1))

        # Stream results
        if opt.view_img:
            cv2.imshow(p, im0)
            if cv2.waitKey(1) == ord('q'):  # q to quit
                raise StopIteration

        # Save results (image with detections)
        if save_img:
            if dataset.mode == 'images':
                cv2.imwrite(save_path, im0)
            else:
                if vid_path != save_path:  # new video
                    vid_path = save_path
                    if isinstance(vid_writer, cv2.VideoWriter):
                        vid_writer.release()  # release previous video writer

                    fourcc = 'mp4v'  # output video codec
                    fps = vid_cap.get(cv2.CAP_PROP_FPS)
                    w = int(vid_cap.get(cv2.CAP_PROP_FRAME_WIDTH))
                    h = int(vid_cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
                    vid_writer = cv2.VideoWriter(save_path, cv2.VideoWriter_fourcc(*fourcc), fps, (w, h))
                vid_writer.write(im0)


image 1/2 /mnt/batch/tasks/shared/LS_root/mounts/clusters/nc6-pm/code/Users/petr.michal/yolov5/data/images/bus.jpg: 640x480 Done. (0.086s)
image 2/2 /mnt/batch/tasks/shared/LS_root/mounts/clusters/nc6-pm/code/Users/petr.michal/yolov5/data/images/zidane.jpg: 384x640 Done. (0.178s)
